In [5]:
# ============================================================
# PyTorch CIFAR Training Script (Full Example)
# Repository: https://github.com/kuangliu/pytorch-cifar
# Model: ResNet (ResNet-18 default)
# Dataset: CIFAR-10
# ============================================================

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import os
import argparse
from torch.utils.data import DataLoader

# ---------------------------
# 1. Konfigurasi Argumen
# ---------------------------
parser = argparse.ArgumentParser(description='PyTorch CIFAR Training')
parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
parser.add_argument('--resume', action='store_true', help='resume from checkpoint')
parser.add_argument('--epochs', default=20, type=int, help='number of total epochs to run')
parser.add_argument('--batch-size', default=128, type=int, help='batch size')
parser.add_argument('--model', default='resnet18', type=str, help='model to use: resnet18, resnet34, resnet50')
args = parser.parse_args(args=[])

# ---------------------------
# 2. Device
# ---------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

# ---------------------------
# 3. Data Augmentation & Loader
# ---------------------------
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), 
                         (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), 
                         (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=args.batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# ---------------------------
# 4. Model
# ---------------------------
from torchvision.models import resnet18, resnet34, resnet50

if args.model == 'resnet18':
    model = resnet18(num_classes=10)
elif args.model == 'resnet34':
    model = resnet34(num_classes=10)
elif args.model == 'resnet50':
    model = resnet50(num_classes=10)
else:
    raise ValueError('Unknown model type')

model = model.to(device)

# ---------------------------
# 5. Loss & Optimizer
# ---------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)

# ---------------------------
# 6. Resume checkpoint jika ada
# ---------------------------
start_epoch = 0
if args.resume and os.path.isfile('checkpoint.pth'):
    print('=> loading checkpoint...')
    checkpoint = torch.load('checkpoint.pth', map_location=device)
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    start_epoch = checkpoint['epoch'] + 1
    print(f'=> loaded checkpoint (epoch {checkpoint["epoch"]})')

# ---------------------------
# 7. Training Loop
# ---------------------------
def train(epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    print(f"Epoch {epoch} | Loss: {running_loss/len(trainloader):.3f} | Acc: {100.*correct/total:.2f}%")

def test(epoch):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0.0
    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    print(f"Test Epoch {epoch} | Loss: {test_loss/len(testloader):.3f} | Acc: {100.*correct/total:.2f}%")

# ---------------------------
# 8. Main Loop
# ---------------------------
for epoch in range(start_epoch, args.epochs):
    train(epoch)
    test(epoch)

    # Simpan checkpoint
    state = {
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'epoch': epoch,
    }
    torch.save(state, 'checkpoint.pth')


Using device: cpu


100%|██████████| 170498071/170498071 [00:43<00:00, 3883730.05it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch 0 | Loss: 2.187 | Acc: 28.70%
Test Epoch 0 | Loss: 1.564 | Acc: 43.12%
Epoch 1 | Loss: 1.521 | Acc: 44.39%
Test Epoch 1 | Loss: 1.426 | Acc: 47.98%
Epoch 2 | Loss: 1.327 | Acc: 51.87%
Test Epoch 2 | Loss: 1.223 | Acc: 56.66%
Epoch 3 | Loss: 1.177 | Acc: 58.04%
Test Epoch 3 | Loss: 1.093 | Acc: 61.93%
Epoch 4 | Loss: 1.058 | Acc: 62.62%
Test Epoch 4 | Loss: 1.422 | Acc: 60.02%
Epoch 5 | Loss: 0.998 | Acc: 64.78%
Test Epoch 5 | Loss: 1.028 | Acc: 65.32%
Epoch 6 | Loss: 0.943 | Acc: 66.96%
Test Epoch 6 | Loss: 0.996 | Acc: 65.73%
Epoch 7 | Loss: 0.899 | Acc: 68.49%
Test Epoch 7 | Loss: 0.873 | Acc: 69.71%
Epoch 8 | Loss: 0.865 | Acc: 70.06%
Test Epoch 8 | Loss: 0.970 | Acc: 67.54%
Epoch 9 | Loss: 0.833 | Acc: 71.05%
Test Epoch 9 | Loss: 0.908 | Acc: 68.97%
Epoch 10 | Loss: 0.808 | Acc: 72.19%
Test Epoch 10 | Loss: 0.913 | Acc: 69.46%
Epoch 11 | Loss: 0.801 | Acc: 72.30%
Test Epoch 11 | Loss: 1.0